In [10]:
!pip install nltk sastrawi
!pip install scikit-learn
!pip install gensim
!pip install matplotlib seaborn wordcloud
!pip install textblob
!pip install tqdm

import pandas as pd
import numpy as np
from datetime import datetime
import json
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from gensim.models import Word2Vec
from textblob import TextBlob
from collections import Counter
import warnings
warnings.filterwarnings('ignore')
from tqdm.auto import tqdm

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [11]:
#load
df = pd.read_csv('/content/dataset_wuwa.csv')
df.head(50)

,username,rating,text,date,thumbsUp,replyContent,replyDate
0,Pengguna Google,5,asik,2025-10-27 18:55:59,0,NaN,NaN
1,Pengguna Google,4,tes game,2025-10-27 16:13:08,0,NaN,NaN
2,Pengguna Google,1,kenapa jadi begini framedrop parah banget,2025-10-27 14:44:39,0,NaN,NaN
3,Pengguna Google,5,full bansos,2025-10-27 14:17:18,0,NaN,NaN
4,Pengguna Google,5,absolute peak cinema ✋😐🤚,2025-10-27 13:33:43,0,NaN,NaN
5,Pengguna Google,5,hold wuthering waves cloud,2025-10-27 12:52:45,0,NaN,NaN
6,Pengguna Google,5,game nya bagus cuy,2025-10-27 12:52:20,0,NaN,NaN
7,Pengguna Google,5,PEAK GAME,2025-10-27 12:15:26,0,NaN,NaN
8,Pengguna Google,5,game nya udah bagus dari story dan musik bagus...,2025-10-27 12:14:12,0,NaN,NaN
9,Pengguna Google,5,hehe luplup,2025-10-27 11:29:32,0,NaN,NaN


In [12]:
#clenaing
df_text = df.copy()

initial_count = len(df_text)
df_text = df_text.drop_duplicates(subset=['text'], keep='first')
print(f"   Duplikat dihapus: {initial_count - len(df_text)}")

df_text = df_text[df_text['text'].notna()]
df_text = df_text[df_text['text'].str.strip() != '']

df_text['word_count'] = df_text['text'].apply(lambda x: len(str(x).split()))
df_text = df_text[df_text['word_count'] >= 3]

df_text = df_text[df_text['word_count'] <= 200]

df_text = df_text.reset_index(drop=True)
print(f"Data bersih: {len(df_text)} baris")

   Duplikat dihapus: 3816
✅ Data bersih: 21419 baris


In [13]:
#presosesing
factory_stemmer = StemmerFactory()
stemmer = factory_stemmer.create_stemmer()
factory_stopword = StopWordRemoverFactory()
stopword_remover = factory_stopword.create_stop_word_remover()

def enhanced_preprocess_text(text):

    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'@\w+|#\w+', '', text)
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    if len(text.split()) < 2:
        return ""

    text = stopword_remover.remove(text)
    text = stemmer.stem(text)

    return text

tqdm.pandas(desc="Processing")
df_text['text_clean'] = df_text['text'].progress_apply(enhanced_preprocess_text)

df_text = df_text[df_text['text_clean'].str.strip() != '']
df_text = df_text.reset_index(drop=True)

print(f"Preprocessing selesai: {len(df_text)} baris")

Processing:   0%|          | 0/21419 [00:00<?, ?it/s]

✅ Preprocessing selesai: 21410 baris


In [27]:
#pelabelan
def improved_sentiment_analysis(text):

    positive_words = [
        'bagus', 'suka', 'mantap', 'keren', 'hebat', 'senang', 'puas', 'terbaik',
        'sempurna', 'lancar', 'cepat', 'mudah', 'jelas', 'menarik', 'seru', 'asyik',
        'nyaman', 'stabil', 'top', 'recommended', 'worth', 'good', 'great', 'amazing',
        'excellent', 'love', 'like', 'best', 'perfect', 'smooth', 'easy', 'nice',
        'terima kasih', 'memuaskan', 'gampang', 'jos', 'luar biasa', 'canggih',
        'mulus', 'responsive', 'indah', 'fantastis', 'oke', 'asik', 'ciamik',
        'juara', 'super', 'optimal', 'efisien', 'cerdas', 'inovatif', 'kreatif'
    ]

    negative_words = [
        'buruk', 'jelek', 'tidak', 'benci', 'marah', 'kecewa', 'payah', 'sampah',
        'bodoh', 'lemot', 'lag', 'error', 'bug', 'crash', 'lambat', 'susah',
        'sulit', 'mahal', 'boros', 'ribet', 'rusak', 'hancur', 'busuk', 'bad',
        'hate', 'worst', 'terrible', 'poor', 'slow', 'broken', 'waste', 'useless',
        'trash', 'disappointing', 'disappointed', 'annoying', 'boring', 'sucks',
        'fail', 'failed', 'gagal', 'mengecewakan', 'berat', 'bingung', 'komplain',
        'ngeselin', 'kesel', 'gak', 'ngga', 'enggak', 'kikir', 'pelit', 'parah',
        'ancur', 'zonk', 'mending', 'nyesel', 'ampas', 'cacat', 'ngelag'
    ]

    text_lower = text.lower()
    words = text_lower.split()

    pos_count = sum(2 if any(pw in word for pw in ['luar biasa', 'sangat bagus', 'terbaik'])
                    else 1 for word in words if any(pw in word for pw in positive_words))
    neg_count = sum(2 if any(nw in word for nw in ['sangat buruk', 'parah banget', 'gagal total'])
                    else 1 for word in words if any(nw in word for nw in negative_words))


    if pos_count > neg_count + 1:
        return 'Positif'
    elif neg_count > pos_count + 1:
        return 'Negatif'
    else:
        return 'Netral'

df_text['sentiment'] = df_text['text_clean'].apply(improved_sentiment_analysis)

print("\nDistribusi sentimen:")
print(df_text['sentiment'].value_counts())

min_samples = df_text['sentiment'].value_counts().min()
if min_samples < 1000:  # Jika ada kelas yang terlalu sedikit
    print("\nWarning: Ada kelas dengan sampel < 1000")
else:
    print(f"Semua kelas memiliki sampel cukup (min: {min_samples})")


Distribusi sentimen:
sentiment
Netral     17129
Positif     3050
Negatif     1231
Name: count, dtype: int64
Semua kelas memiliki sampel cukup (min: 1231)


In [28]:
#split data encoding
le = LabelEncoder()
df_text['sentiment_encoded'] = le.fit_transform(df_text['sentiment'])

X = df_text['text_clean']
y = df_text['sentiment_encoded']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training: {len(X_train)} | Testing: {len(X_test)}")

Training: 17128 | Testing: 4282


In [29]:
#TF-IDF + Split 80/20
X_train1, X_test1, y_train1, y_test1 = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"   Training: {len(X_train1):,} | Testing: {len(X_test1):,}")

tfidf1 = TfidfVectorizer(
    max_features=10000,
    ngram_range=(1,3),
    min_df=2,
    max_df=0.95,
    sublinear_tf=True
)
X_train_tfidf1 = tfidf1.fit_transform(X_train1)
X_test_tfidf1 = tfidf1.transform(X_test1)

print(f"   Shape fitur: {X_train_tfidf1.shape}")

svm_model = SVC(
    kernel='linear',
    C=5.0,
    random_state=42,
    class_weight='balanced'
)
svm_model.fit(X_train_tfidf1, y_train1)

y_train_pred1 = svm_model.predict(X_train_tfidf1)
y_test_pred1 = svm_model.predict(X_test_tfidf1)

train_acc1 = accuracy_score(y_train1, y_train_pred1)
test_acc1 = accuracy_score(y_test1, y_test_pred1)
train_f1_1 = f1_score(y_train1, y_train_pred1, average='weighted')
test_f1_1 = f1_score(y_test1, y_test_pred1, average='weighted')

print(f"\n   hasil (Optimized):")
print(f"      Train Accuracy: {train_acc1}")
print(f"      Test Accuracy:  {test_acc1}")

   Training: 17,128 | Testing: 4,282
   Shape fitur: (17128, 10000)

   hasil (Optimized):
      Train Accuracy: 0.9804997664642691
      Test Accuracy:  0.864549276039234


In [30]:
# SKEMA 2: Random Forest + TF-IDF + Word2Vec + 80/20 (Enhanced)
print("\n🔹 SKEMA 2: Random Forest + TF-IDF + Word2Vec + Split 80/20 (Enhanced)")
print("-" * 70)

# TF-IDF dengan ngram lebih luas dan max_features lebih besar
tfidf2 = TfidfVectorizer(
    max_features=8000,
    ngram_range=(1,3),
    min_df=2,
    max_df=0.95,
    sublinear_tf=True
)
X_train_tfidf2 = tfidf2.fit_transform(X_train2)
X_test_tfidf2 = tfidf2.transform(X_test2)

# Word2Vec dengan vector lebih besar dan window lebih luas
w2v_model2 = Word2Vec(
    sentences=X_train2_tokens,
    vector_size=300,
    window=10,
    min_count=1,
    workers=4,
    sg=1,
    epochs=40,
    seed=42
)
print(f"   Vocabulary size: {len(w2v_model2.wv):,}")

# Convert ke vektor rata-rata + normalisasi
def text_to_w2v2(tokens, model):
    vectors = [model.wv[token] for token in tokens if token in model.wv]
    if len(vectors) > 0:
        vec = np.mean(vectors, axis=0)
        return vec / np.linalg.norm(vec)  # normalisasi
    else:
        return np.zeros(model.vector_size)

X_train_w2v2 = np.array([text_to_w2v2(t, w2v_model2) for t in X_train2_tokens])
X_test_w2v2 = np.array([text_to_w2v2(t, w2v_model2) for t in X_test2_tokens])
print(f"   Shape fitur: {X_train_w2v2.shape}")

# Gabungkan TF-IDF + Word2Vec
from scipy.sparse import hstack
X_train_combined2 = hstack([X_train_tfidf2, X_train_w2v2])
X_test_combined2 = hstack([X_test_tfidf2, X_test_w2v2])

# Random Forest lebih kuat
rf_model2 = RandomForestClassifier(
    n_estimators=600,
    max_depth=45,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features='sqrt',
    random_state=42,
    n_jobs=-1,
    class_weight='balanced'
)
rf_model2.fit(X_train_combined2, y_train2)

# Evaluasi
y_train_pred2 = rf_model2.predict(X_train_combined2)
y_test_pred2 = rf_model2.predict(X_test_combined2)

train_acc2 = accuracy_score(y_train2, y_train_pred2)
test_acc2 = accuracy_score(y_test2, y_test_pred2)
train_f1_2 = f1_score(y_train2, y_train_pred2, average='weighted')
test_f1_2 = f1_score(y_test2, y_test_pred2, average='weighted')

print(f"\n   ✅ HASIL SKEMA 2 (Enhanced TF-IDF + Word2Vec):")
print(f"      Train Accuracy: {train_acc2*100:.2f}%")
print(f"      Test Accuracy:  {test_acc2*100:.2f}%")
print(f"      Train F1-Score: {train_f1_2:.4f}")
print(f"      Test F1-Score:  {test_f1_2:.4f}")

results.append({
    'Skema': 'Skema 2',
    'Algoritma': 'Random Forest',
    'Ekstraksi Fitur': 'TF-IDF + Word2Vec',
    'Split Data': '80/20',
    'Train Accuracy (%)': train_acc2 * 100,
    'Test Accuracy (%)': test_acc2 * 100,
    'Train F1-Score': train_f1_2,
    'Test F1-Score': test_f1_2
})



🔹 SKEMA 2: Random Forest + TF-IDF + Word2Vec + Split 80/20 (Enhanced)
----------------------------------------------------------------------
   Vocabulary size: 13,801
   Shape fitur: (17128, 300)

   ✅ HASIL SKEMA 2 (Enhanced TF-IDF + Word2Vec):
      Train Accuracy: 100.00%
      Test Accuracy:  81.62%
      Train F1-Score: 1.0000
      Test F1-Score:  0.7525


In [31]:
# SKEMA 3: Random Forest + TF-IDF + 70/30
# ===================================================================
print("\n🔹 SKEMA 3: Random Forest + TF-IDF + Split 70/30")
print("-" * 70)

# Split 70/30
X_train3, X_test3, y_train3, y_test3 = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"   Training: {len(X_train3):,} | Testing: {len(X_test3):,}")

# TF-IDF
print("   Ekstraksi fitur TF-IDF...")
tfidf3 = TfidfVectorizer(
    max_features=8000,
    ngram_range=(1, 3),
    min_df=2,
    max_df=0.95,
    sublinear_tf=True
)
X_train_tfidf3 = tfidf3.fit_transform(X_train3)
X_test_tfidf3 = tfidf3.transform(X_test3)

print(f"   Shape fitur: {X_train_tfidf3.shape}")

# Training Random Forest
print("   Training Random Forest...")
rf_model3 = RandomForestClassifier(
    n_estimators=200,
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=2,
    max_features='sqrt',
    random_state=42,
    n_jobs=-1,
    class_weight='balanced'
)
rf_model3.fit(X_train_tfidf3, y_train3)

# Evaluasi
y_train_pred3 = rf_model3.predict(X_train_tfidf3)
y_test_pred3 = rf_model3.predict(X_test_tfidf3)

train_acc3 = accuracy_score(y_train3, y_train_pred3)
test_acc3 = accuracy_score(y_test3, y_test_pred3)
train_f1_3 = f1_score(y_train3, y_train_pred3, average='weighted')
test_f1_3 = f1_score(y_test3, y_test_pred3, average='weighted')

print(f"\n   ✅ HASIL SKEMA 3:")
print(f"      Train Accuracy: {train_acc3*100:.2f}%")
print(f"      Test Accuracy:  {test_acc3*100:.2f}%")
print(f"      Train F1-Score: {train_f1_3:.4f}")
print(f"      Test F1-Score:  {test_f1_3:.4f}")

results.append({
    'Skema': 'Skema 3',
    'Algoritma': 'Random Forest',
    'Ekstraksi Fitur': 'TF-IDF',
    'Split Data': '70/30',
    'Train Accuracy (%)': train_acc3 * 100,
    'Test Accuracy (%)': test_acc3 * 100,
    'Train F1-Score': train_f1_3,
    'Test F1-Score': test_f1_3
})


🔹 SKEMA 3: Random Forest + TF-IDF + Split 70/30
----------------------------------------------------------------------
   Training: 14,987 | Testing: 6,423
   Ekstraksi fitur TF-IDF...
   Shape fitur: (14987, 8000)
   Training Random Forest...

   ✅ HASIL SKEMA 3:
      Train Accuracy: 80.83%
      Test Accuracy:  74.67%
      Train F1-Score: 0.8252
      Test F1-Score:  0.7718


In [ ]:
# 8. DETERMINE BEST MODEL
# ==========================================
print("\n[STEP 8/8] Menentukan model terbaik...")

best_idx = df_results['Test Accuracy (%)'].idxmax()
best_skema = df_results.loc[best_idx]

print("\n" + "="*70)
print("🏆 MODEL TERBAIK")
print("="*70)
print(f"   Skema:            {best_skema['Skema']}")
print(f"   Algoritma:        {best_skema['Algoritma']}")
print(f"   Ekstraksi Fitur:  {best_skema['Ekstraksi Fitur']}")
print(f"   Split Data:       {best_skema['Split Data']}")
print(f"   Test Accuracy:    {best_skema['Test Accuracy (%)']:.2f}%")
print(f"   Test F1-Score:    {best_skema['Test F1-Score']:.4f}")
print("="*70)

# Detailed report for best model
print(f"\n📊 Classification Report ({best_skema['Skema']}):")
if best_idx == 0:
    print(classification_report(y_test1, y_test_pred1, target_names=le.classes_))
    cm = confusion_matrix(y_test1, y_test_pred1)
elif best_idx == 1:
    print(classification_report(y_test2, y_test_pred2, target_names=le.classes_))
    cm = confusion_matrix(y_test2, y_test_pred2)
else:
    print(classification_report(y_test3, y_test_pred3, target_names=le.classes_))
    cm = confusion_matrix(y_test3, y_test_pred3)


In [ ]:
#simpan file

In [ ]:
#file requirement txt